### Problem Statement

Starting from 'IKinBody' in the MR code library, write a new function, `IKinBodyIterates`. This function prints out a report for each interation of the Newton-Raphson process, for iterates 0 (the initial guess) to the final answer. Each iteration reports the iteration number _i_, the joint vector $ \theta^i $, the end-effector configuration $T_{sb}(\theta^i)$. the error twist $\Upsilon_b$, and the angular and linear error ||$\omega_b$|| and ||$\upsilon_b$|| (something like the able at the end of Chapter 6.2.2). For a four-joint robot, a typical iterate might look like:


`Iteration 3:`

`joint vector:`
`0.221, 0.375, 2.233, 1.414`

`SE(3) end-effector config:`
`1.000  0.000  0.000  3.275`
`0.000  1.000  0.000  4.162`
`0.000  0.000  1.000 -5.732`
`    0      0      0      1` 

`          error twist V_b: (0.232, 0.171, 0.211, 0.345, 1.367, -0.222)`
`angular error ||omega_b||:  0.357`
`linear error ||v_b||: 1.427`

The function should also save the joint vector of each iteration as a row in a matrix.

### Approach
This problem asks us to build off of the modern_robotics function `IKinBody` so the first step should be to read the documentation to source this function and understand how it works. The code is then commented for understandibility, and tested to make sure that the porting process of the function does not affect its performance from a native Python setting to a Jupyter notebook. 

In [1]:
#import necessary packages
import modern_robotics as mr 
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
#our starting point for this function should be IKinBody
def IKinBody(Blist, M, T, thetalist0, eomg, ev):
    """Computes inverse kinematics in the body frame for an open chain robot

    :param Blist: The joint screw axes in the end-effector frame when the
                  manipulator is at the home position, in the format of a
                  matrix with axes as the columns
    :param M: The home configuration of the end-effector
    :param T: The desired end-effector configuration Tsd
    :param thetalist0: An initial guess of joint angles that are close to
                       satisfying Tsd
    :param eomg: A small positive tolerance on the end-effector orientation
                 error. The returned joint angles must give an end-effector
                 orientation error less than eomg
    :param ev: A small positive tolerance on the end-effector linear position
               error. The returned joint angles must give an end-effector
               position error less than ev
    :return thetalist: Joint angles that achieve T within the specified
                       tolerances,
    :return success: A logical value where TRUE means that the function found
                     a solution and FALSE means that it ran through the set
                     number of maximum iterations without finding a solution
                     within the tolerances eomg and ev.
    Uses an iterative Newton-Raphson root-finding method.
    The maximum number of iterations before the algorithm is terminated has
    been hardcoded in as a variable called maxiterations. It is set to 20 at
    the start of the function, but can be changed if needed.

    Example Input:
        Blist = np.array([[0, 0, -1, 2, 0,   0],
                          [0, 0,  0, 0, 1,   0],
                          [0, 0,  1, 0, 0, 0.1]]).T
        M = np.array([[-1, 0,  0, 0],
                      [ 0, 1,  0, 6],
                      [ 0, 0, -1, 2],
                      [ 0, 0,  0, 1]])
        T = np.array([[0, 1,  0,     -5],
                      [1, 0,  0,      4],
                      [0, 0, -1, 1.6858],
                      [0, 0,  0,      1]])
        thetalist0 = np.array([1.5, 2.5, 3])
        eomg = 0.01
        ev = 0.001
    Output:
        (np.array([1.57073819, 2.999667, 3.14153913]), True)
    """

    #Make an np.array copy of the input thetatlist. Perhaps for flexibility to permit lists an input entries? 
    thetalist = np.array(thetalist0).copy()

    #Initialize the counter variable
    i = 0

    #Hardcode the number of iterations that we want to do. Should probably make this an input variable. 
    maxiterations = 20

    #This is the error twist. 
    Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M, Blist, thetalist)), T)))

    #Evaluates true if either of them is larger than the specified tolerance limit. 
    err = np.linalg.norm([Vb[0], Vb[1], Vb[2]]) > eomg or np.linalg.norm([Vb[3], Vb[4], Vb[5]]) > ev
  
  
    while err and i < maxiterations:

        #This updates the thetalist from 
        thetalist = thetalist + np.dot(np.linalg.pinv(mr.JacobianBody(Blist, thetalist)), Vb)

        #update our iteration. Mainly used to make sure that we are under the maxiteration limit. 
        i = i + 1

        #Re-evaluate the error twist. 
        Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M, Blist, thetalist)), T)))

        #Re-evaluates the truthfulness of this statement to see if the function is meeting tolerances. 
        err = np.linalg.norm([Vb[0], Vb[1], Vb[2]]) > eomg or np.linalg.norm([Vb[3], Vb[4], Vb[5]]) > ev

    #Returns the final thetalist at the end position, as well if we met the tolerances that we were given. 
    return (thetalist, not err)

In [3]:
#To test the code, we use the example case in the code documentation. 
Blist = np.array([[0, 0, -1, 2, 0,   0],
                  [0, 0,  0, 0, 1,   0],
                  [0, 0,  1, 0, 0, 0.1]]).T

M = np.array([[-1, 0,  0, 0],
              [ 0, 1,  0, 6],
              [ 0, 0, -1, 2],
              [ 0, 0,  0, 1]])

T = np.array([[0, 1,  0,     -5],
              [1, 0,  0,      4],
              [0, 0, -1, 1.6858],
              [0, 0,  0,      1]])

thetalist0 = np.array([1.5, 2.5, 3])
eomg = 0.01
ev = 0.001      

expectedans = (np.array([1.57073819, 2.999667, 3.14153913]), True)

#Easier evaluation would be using the equality operator but precision issues render the first half of the code dissimilar.
print(expectedans)
print(IKinBody(Blist, M, T, thetalist0, eomg, ev))


(array([1.57073819, 2.999667  , 3.14153913]), True)
(array([1.57073819, 2.999667  , 3.14153913]), True)


In [56]:
#Completing the assignment as specified.
def IKinBodyIterates(Blist, M, T, thetalist0, eomg, ev):
    """Computes inverse kinematics in the body frame for an open chain robot

    :param Blist: The joint screw axes in the end-effector frame when the
                  manipulator is at the home position, in the format of a
                  matrix with axes as the columns
    :param M: The home configuration of the end-effector
    :param T: The desired end-effector configuration Tsd
    :param thetalist0: An initial guess of joint angles that are close to
                       satisfying Tsd
    :param eomg: A small positive tolerance on the end-effector orientation
                 error. The returned joint angles must give an end-effector
                 orientation error less than eomg
    :param ev: A small positive tolerance on the end-effector linear position
               error. The returned joint angles must give an end-effector
               position error less than ev
    :return thetalist: Joint angles that achieve T within the specified
                       tolerances,
    :return success: A logical value where TRUE means that the function found
                     a solution and FALSE means that it ran through the set
                     number of maximum iterations without finding a solution
                     within the tolerances eomg and ev.
    Uses an iterative Newton-Raphson root-finding method.
    The maximum number of iterations before the algorithm is terminated has
    been hardcoded in as a variable called maxiterations. It is set to 20 at
    the start of the function, but can be changed if needed.

    Example Input:
        Blist = np.array([[0, 0, -1, 2, 0,   0],
                          [0, 0,  0, 0, 1,   0],
                          [0, 0,  1, 0, 0, 0.1]]).T
        M = np.array([[-1, 0,  0, 0],
                      [ 0, 1,  0, 6],
                      [ 0, 0, -1, 2],
                      [ 0, 0,  0, 1]])
        T = np.array([[0, 1,  0,     -5],
                      [1, 0,  0,      4],
                      [0, 0, -1, 1.6858],
                      [0, 0,  0,      1]])
        thetalist0 = np.array([1.5, 2.5, 3])
        eomg = 0.01
        ev = 0.001
    Output:
        (np.array([1.57073819, 2.999667, 3.14153913]), True)
    """

    #Make an np.array copy of the input thetatlist. Perhaps for flexibility to permit lists an input entries? 
    thetalist = np.array(thetalist0).copy()

    #create a global list of all the theta vector iterates. The initial guess will be initialized into the array.
    thetaEval = np.array(thetalist0[None, :])

    #Initialize the counter variable
    i = 0

    #Hardcode the number of iterations that we want to do. Should probably make this an input variable. 
    maxiterations = 20

    #This is the error twist. 
    Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M, Blist, thetalist)), T)))

    #Evaluates true if either of them is larger than the specified tolerance limit. 
    err = np.linalg.norm([Vb[0], Vb[1], Vb[2]]) > eomg or np.linalg.norm([Vb[3], Vb[4], Vb[5]]) > ev
    errAcc = np.array([np.linalg.norm([Vb[0], Vb[1], Vb[2]]),np.linalg.norm([Vb[3], Vb[4], Vb[5]])]).reshape(1,2)
  
  
    while err and i < maxiterations:

        #This updates the thetalist and adds it to the global thetalist.
        thetalist = thetalist + np.dot(np.linalg.pinv(mr.JacobianBody(Blist, thetalist)), Vb)
        thetaEval = np.concatenate((thetaEval, thetalist[None, :]), axis = 0)

        #update our iteration. Mainly used to make sure that we are under the maxiteration limit. 
        i = i + 1

        #Re-evaluate the error twist. 
        Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M, Blist, thetalist)), T)))

        #Re-evaluates the truthfulness of this statement to see if the function is meeting tolerances. 
        err = np.linalg.norm([Vb[0], Vb[1], Vb[2]]) > eomg or np.linalg.norm([Vb[3], Vb[4], Vb[5]]) > ev
        errAcc = np.concatenate((errAcc, np.array([np.linalg.norm([Vb[0], Vb[1], Vb[2]]),np.linalg.norm([Vb[3], Vb[4], Vb[5]])]).reshape(1,2)), axis = 0)

        #Print the iteration that we are on. 
        iterStr = f"Iteration: {i} \n\r"
        jointVec = f"joint vector: \n\r {thetalist} \n\n\r"
        endEffec = f"SE(3) end-effector config: \n\r {mr.FKinBody(M, Blist, thetalist)} \n\n\r"
        errTwist = f"error twist V_b: {Vb} \n\r"
        angError = f"angular error ||omega_b||: {np.linalg.norm([Vb[0], Vb[1], Vb[2]])} \n\r"
        linError = f"linear error ||v_b||: {np.linalg.norm([Vb[3], Vb[4], Vb[5]])}\n\r"

        print(iterStr, jointVec, endEffec, errTwist, angError, linError)

    #Save the final thetaEval evolution matrix. 
    np.savetxt('thetaEvolution' + f'{i}'+'.csv', thetaEval, delimiter=',')

    #Returns the final thetalist at the end position, as well if we met the tolerances that we were given. 
    return (errAcc, thetaEval, thetalist, not err)

In [60]:
#Easier evaluation would be using the equality operator but precision issues render the first half of the code dissimilar.
# print(IKinBodyIterates(Blist, M, T, thetalist0, eomg, ev))
# print(expectedans)                                                             

In [61]:
#Define the necessary variables. 
L1 = 0.425
L2 = 0.392
H1 = 0.089
H2 = 0.095
W1 = 0.109
W2 = 0.082

Blist = np.array([[0,  1,  0, W1 + W2,        0, L1 + L2],
                  [0,  0,  1,      H2, -L1 - L2,       0],
                  [0,  0,  1,      H2,      -L2,       0],
                  [0,  0,  1,      H2,        0,       0],
                  [0, -1,  0,     -W2,        0,       0],
                  [0,  0,  1,       0,        0,       0]]).T

M = np.array([[-1, 0,  0, L1 + L2],
              [ 0, 0,  1, W1 + W2],
              [ 0, 1,  0, H1 - H2],
              [ 0, 0,  0,       1]])

T = np.array([[0.7071, 0,  0.7071, -0.3],
              [0.7071, 0, -0.7071, -0.5],
              [     0, 1,       0,  0.5],
              [     0, 0,       0,    1]])
 
#first guess never converges
thetalist01 = np.array([0, 0, 0, 0, 0, 0])
#second guess converges in 3 
thetalist0 = np.array([-5, 3, -5, -1, 2.5, 2.6]) 
eomg = 0.001
ev = 0.0001      

#Easier evaluation would be using the equality operator but precision issues render the first half of the code dissimilar.
# print(IKinBody(Blist, M, T, thetalist0, eomg, ev))
e, a, b, c = IKinBodyIterates(Blist, M, T, thetalist0, eomg, ev)

print("\n\n\rStart of the new guess.\n\n\r")


e01, a01, b01, c01 = IKinBodyIterates(Blist, M, T, thetalist01, eomg, ev)


Iteration: 1 
 joint vector: 
 [-5.17853868  2.98496164 -4.83238996 -1.30941287  2.79673719  3.1491369 ] 

 SE(3) end-effector config: 
 [[ 0.72485836 -0.01232199  0.68878772 -0.27833773]
 [ 0.68854996 -0.01881614 -0.72494476 -0.48239567]
 [ 0.02189309  0.99974703 -0.00515473  0.4943664 ]
 [ 0.          0.          0.          1.        ]] 

 error twist V_b: [-0.00487433  0.0256252  -0.02195976 -0.02773804  0.00593166 -0.00252772] 
 angular error ||omega_b||: 0.03409752319315265 
 linear error ||v_b||: 0.02847758455208408

Iteration: 2 
 joint vector: 
 [-5.20216168  3.01629094 -4.94107537 -1.21806957  2.84594514  3.14021905] 

 SE(3) end-effector config: 
 [[ 7.07122160e-01  3.77939056e-04  7.07091301e-01 -3.00547132e-01]
 [ 7.07091382e-01 -1.41795953e-04 -7.07122166e-01 -4.98818973e-01]
 [-1.66986399e-04  9.99999919e-01 -3.67504510e-04  4.98550913e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

 error twist V_b: [-3.67505752e-04  2.17941718e-05  1.66982132

In [21]:
%matplotlib qt

#We have an array of theta variables as the function iterates. Create a position matrix.
posMat = np.zeros((a.shape[0],3))
posMat1 = np.zeros((a01.shape[0],3))


#Find the SE(3) matrix for each angle position. 
for i in range(a.shape[0]):

    #get the current T from the forward kinematics
    currT = mr.FKinBody(M, Blist, a[i, :])

    #extract the x,y,z position from the T
    posMat[i,:] = currT[0:3,-1]

#Find the SE(3) matrix for each angle position. 
for i in range(a01.shape[0]):

    #get the current T from the forward kinematics
    currT1 = mr.FKinBody(M, Blist, a01[i, :])

    #extract the x,y,z position from the T
    posMat1[i,:] = currT1[0:3,-1]

#Plot the position matrix of the short matrix.
ax = plt.figure().add_subplot(projection='3d')

#Plot the x,y,z positions of the arm as it moves.
ax.plot(posMat[:,0],posMat[:,1],posMat[:,2], label="Short")
ax.plot(posMat1[:,0],posMat1[:,1],posMat1[:,2], label="Long")

#plot the start and end positions as points. 
ax.plot(posMat[0,0],posMat[0,1],posMat[0,2], color="blue", marker = "v", label= "Start")
ax.plot(posMat1[0,0],posMat1[0,1],posMat1[0,2], color="darkorange", marker = "v", label= "Start")

ax.plot(posMat[-1,0],posMat[-1,1],posMat[-1,2], color="blue", marker = "*", label= "End")
ax.plot(posMat1[-1,0],posMat1[-1,1],posMat1[-1,2], color="darkorange", marker = "*", label= "End")

ax.legend()

#Set some axis titles 
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('Initial Guess to Convergence')

plt.show()


In [41]:
#Create another graph for the linear and angular error as a function of iteration times. 
fig, axs = plt.subplots(1,2)
axs[0].set_title("Angular Error")
axs[1].set_title( "Linear Error")

axs[0].set_xlabel("Iterations")
axs[1].set_xlabel("Iterations")

axs[0].set_ylabel("Error")
axs[1].set_ylabel("Error")

axs[0].plot(e[:,0]  , color="crimson")
axs[1].plot(e[:,1], color="crimson")

axs[0].plot(e01[:, 0]  , color="blueviolet")
axs[1].plot(e01[:, 1], color="blueviolet")


plt.show()